Aqui está um código simples com sklearn para testar a execução de Python no Codespace com uma Decision Tree:

In [55]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [56]:
def carregar_dataset(dataset):
    """
    Descrição:
        Essa função é usada para carregar um dataset já separando em dados e label

    parametros: dataset
        representa qual dataset será usado no carregar_dataset

    retorno: dataset, X, Y
        dataset = representa todos os valores do dataset
        x = representa apenas os valores do dataset
        y = representa apenas as labels do dataset

    """
    # Carregar o dataset especifico
    dataset_carregado = dataset()
    x = dataset_carregado.data
    y = dataset_carregado.target

    return dataset_carregado, x, y

In [57]:
def dividir_treinamento(data, label, test_size, random_state):
    """"
    Descrição: 
        Função usada para Dividir matrizes ou matrizes em subconjuntos de trem aleatório e teste. para isso vou usar a função train_test_split()
        
    Parametrôs: data, label, test_size, random_state
        data = Os dados do dataset pode ser também o X
        label = são o processo usado para identificar as catégorias pode ser também o Y
        test_size = representar a proporção do conjunto de dados para incluir na divisão de teste.
        random_state = Controla o embaralhamento aplicado aos dados antes de aplicar a divisão. Passe um int para saída reprodutível em várias chamadas de função.

    Retorno: X_train, X_test, y_train, y_test
        X_train = dados de treinamento separado
        X_test = dados de teste separado
        y_train = label para o treinamento separado
        y_test = label para o test separado
    """
    # Dividir o dataset em treinamento e teste
    X_train, X_test, y_train, y_test = train_test_split(data, label, test_size, random_state)

    return X_train, X_test, y_train, y_test

In [58]:
def treinando_modelo(modelo, X_train, y_train, X_test, random_state):
    """
    Descrição:
        Função para treinar um modelo de classificação usando o sklearn
    
    Parametrôs:
        modelo = colocamos o modelo que queremos usar como classificador
        X_train = dados de treinamento separado
        X_test = dados de teste separado
        y_train = label para o treinamento separado
        random_state = Controla o embaralhamento aplicado aos dados antes de aplicar a divisão. Passe um int para saída reprodutível em várias chamadas de função.

    Retorno:
        modelo_treinado = o treinamento feito
        y_pred = classe que foi prevista no conjunto de teste
    """
    # Criar um modelo
    modelo_treinado = modelo(random_state)

    # Treinar a Decision Tree
    modelo_treinado.fit(X_train, y_train)

    # Prever as classes para o conjunto de teste
    y_pred = modelo_treinado.predict(X_test)

    return modelo_treinado, y_pred


In [59]:
data, X, Y = carregar_dataset(load_iris)
X_train, X_test, y_train, y_test = dividir_treinamento(X, Y, test_size=0.2, random_state=42)
#modelo_treinado, y_pred = treinando_modelo(X_train, y_train, X_test, modelo=DecisionTreeClassifier, random_state=42)
#print("Precisão:", modelo_treinado.score(X_test, y_test))

TypeError: Singleton array array(0.2) cannot be considered a valid collection.